In [1]:
sc.master

'local[*]'

In [9]:
from pyspark.ml.linalg import Vectors,Vector

In [11]:
data = sc.textFile("inputData/iris.txt").map(lambda x:x.split(",")).map(lambda x:(Vectors.dense(float(x[0]),float(x[1]),float(x[2]),float(x[3])),x[4]))

In [22]:
df = data.toDF()
df.show()

+-----------------+-----------+
|               _1|         _2|
+-----------------+-----------+
|[5.1,3.5,1.4,0.2]|Iris-setosa|
|[4.9,3.0,1.4,0.2]|Iris-setosa|
|[4.7,3.2,1.3,0.2]|Iris-setosa|
|[4.6,3.1,1.5,0.2]|Iris-setosa|
|[5.0,3.6,1.4,0.2]|Iris-setosa|
|[5.4,3.9,1.7,0.4]|Iris-setosa|
|[4.6,3.4,1.4,0.3]|Iris-setosa|
|[5.0,3.4,1.5,0.2]|Iris-setosa|
|[4.4,2.9,1.4,0.2]|Iris-setosa|
|[4.9,3.1,1.5,0.1]|Iris-setosa|
|[5.4,3.7,1.5,0.2]|Iris-setosa|
|[4.8,3.4,1.6,0.2]|Iris-setosa|
|[4.8,3.0,1.4,0.1]|Iris-setosa|
|[4.3,3.0,1.1,0.1]|Iris-setosa|
|[5.8,4.0,1.2,0.2]|Iris-setosa|
|[5.7,4.4,1.5,0.4]|Iris-setosa|
|[5.4,3.9,1.3,0.4]|Iris-setosa|
|[5.1,3.5,1.4,0.3]|Iris-setosa|
|[5.7,3.8,1.7,0.3]|Iris-setosa|
|[5.1,3.8,1.5,0.3]|Iris-setosa|
+-----------------+-----------+
only showing top 20 rows



In [23]:
df = df.select(df["_1"].alias("features"),df["_2"].alias("label"))

In [29]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression

In [28]:
labelIndex = StringIndexer(inputCol="label",outputCol="indexlabel").fit(df)

In [30]:
model = LogisticRegression(labelCol="indexlabel",featuresCol="features")

In [31]:
from pyspark.ml import Pipeline

In [32]:
from pyspark.ml.feature import IndexToString

In [33]:
labelConvert = IndexToString(inputCol="prediction",outputCol="predictionLable",labels=labelIndex.labels)

In [34]:
pipeline = Pipeline(stages=[labelIndex,model,labelConvert])

In [35]:
train_df,test_df = df.randomSplit([0.7,0.3])

In [36]:
pipelineModel = pipeline.fit(train_df)

In [37]:
prediction = pipelineModel.transform(test_df)

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [40]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="indexlabel",metricName="accuracy")

In [41]:
accurcy = evaluator.evaluate(prediction)
print(accurcy)

0.9736842105263158


In [43]:
model = pipelineModel.stages[1]

In [46]:
print(model.coefficientMatrix)
print(model.interceptVector)

DenseMatrix([[  3489.45911926, -10999.9633708 ,   1411.84847119,
                2491.48571394],
             [  2704.36769591, -13385.69480002,   4541.71455776,
               10939.47068188],
             [ -6193.82681517,  24385.65817082,  -5953.56302895,
              -13430.95639581]])
[8873.83062042813,-9471.02731974555,597.1966993174196]
